# Kildomaten

*Kildomaten* er tjeneste som er tilgjenglig for alle Dapla-team. Den lar **data-admins** automatisere prosesseringen fra **kildedata** til **inndata**. Det skjer ved brukeren skriver et python-skript som skal prosessere alle filer som kommer inn i en gitt mappe i kildedata-bøtta. Brukeren bestemmer selv hvilken undermappe i bøtta som tjenesten skal trigge på.

Prosessering som skal skje i overgangen fra kildedata til inndata har SSB definert som:

- **Dataminimering**:  
Fjerne alle felter som ikke er strengt nødvendig for å produsere statistikk.
- **Pseudonymisering**:  
Pseudonymisering av personidentifiserende data.
- **Kodeverk**:  
Legge på standard kodeverk fra for eksempel [Klass](https://www.ssb.no/klass/).  
- **Standardisering**:  
Tegnsett, datoformat, etc. endres til SSBs standardformat.

Under forklarer vi nærmere hvordan man bruker tjenesten. 

## Forberedelser

Før et Dapla-team kan ta i bruk *Kildomaten* må man koble teamets GitHub-IaC-repo til kildeprosjektet [slik som beskrevet her](./github-app-integrasjon.html). Det er en engangsjobb som må gjøre av teamets **data-admin**.

## Sette opp tjenesten

I denne delen bryter vi ned prosessen med å sette opp *kildomaten* i de stegene vi mener det er hensiktsmessig å gjøre det når den settes opp for første gang.

### Klone IaC-repoet

Siden vi skal endre på teamets IaC-repo så må vi først klone ned repoet. Man finner teamets IaC-repo ved gå inn på [SSBs GitHub-organisasjon](https://github.com/statisticsnorway) og søke etter repoet som heter `<teamnavn>-iac`. Når du har funnet repoet så kan du gjøre følgende:

1. Klon ned ditt teams Iac-repo `git clone <repo-url>`
2. Opprett en ny branch: `git checkout -b add-kildomaten-source`

### Opprett mappestruktur

For at *Kildomaten* skal fungere så må vi følge en bestemt mappestruktur i IaC-repoet. Denne strukturen er nødvendig for at tjenesten skal vite hvor den skal hente kildedata fra, og hvor den skal legge inndata. Denne strukturen blir ikke laget ved opprettelsen av teamet, siden ikke alle team kommer til å bruke tjenesten. Vi kan derfor starte med å opprette denne strukturen. Den skal se slik ut:

```{.yaml filename="github.com/statisticsnorway/<teamnavn>-iac"}
dapla-example-iac
├── automation/
│   └── source_data/
│       ├── prod/
│       │   
│       └── test/
│...           
```

Husk at du kun skal opprette mappen som heter `automation` og dets undermapper. Resten av mappene skal allerede være opprettet og du skal ikke endre de.

### Velg trigger-mappe

Siden tjenesten kan trigges på filer som skrives til undermapper i en bøtte, og ulike undermapper kan trigge ulike python-skript, så er det viktig å tenke gjennom hvordan man organiserer filer i bøtta. 

:::: {.columns}

::: {.column width="47%"}
```{.yaml filename="bøtte"}
dapla-example-iac
├── automation/
│   └── source_data/
│       ├── prod/
│       │   
│       └── test/
│...           
```
:::

::: {.column width="6%"}

:::

::: {.column width="47%"}

```{.yaml filename="repo"}
dapla-example-iac
├── automation/
│   └── source_data/
│       ├── prod/
│       │   
│       └── test/
│...           
```
:::


::::

## Vedlikehold

dkjfkl


